In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate
from path import Path

In [ ]:
#verify and remove invalid images
path = Path('/path/to/artists/')
for directory in os.listdir(path):
  if os.path.isdir(path/directory):
    verify_images(path/directory)

In [ ]:
#create databunch
#set batch size, validation percent, image size, use default transforms and normalize using imagenet stats
bs = 64
data = ImageDataBunch.from_folder(path, valid_pct = .2, ds_tfms=get_transforms(), size=224, bs=bs
                                 ).normalize(imagenet_stats)

In [ ]:
#preview data
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
#create resnet34 CNN
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
#fit for 4 epochs
learn.fit_one_cycle(4)

In [ ]:
#save
learn.save('stage-1')

In [ ]:
#get losses and verify that len is equal to len of validation dataset
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
#find and plot learning rate
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
#unfreeze pretrained layers and fit for 4 epochs using range of learning rates from lr finder
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-5,1e-3))